In [2]:
import numpy as np
from scipy.linalg import orth

def dxdt(x,model_params):
    # Define the function here
    return x
def dxdt_tan(x,dx,model_params):
    # Define the tangent map here
    return dx
def dxdt_lorenz63(x, sigma = 10., beta = 8/3, rho = 28.):
    return np.array([sigma*(- x[0] + x[1]),\
                     rho*x[0] - x[1] - x[0]*x[2],\
                     x[0]*x[1]-beta*x[2]])
def dxdt_lorenz63_tan(x, dx, sigma = 10., beta = 8/3, rho = 28.):
    return np.array([sigma*(- dx[0] + dx[1]),\
                     rho*dx[0] - x[0]*dx[2] - dx[0]*x[2] - dx[1],\
                     dx[0]*x[1]+x[0]*dx[1]-beta*dx[2]])
def rk4_integrator(x,tau,dxdt):
    # 4th order runge-kutta integrator for systems with no explicit time dependence
    k1 = dxdt(x)
    k2 = dxdt(x + k1/2*tau)
    k3 = dxdt(x + k2/2*tau)
    k4 = dxdt(x + tau*k3)
    
    xnext = x + 1/6*tau*(k1+2*k2+2*k3+k4)
    return xnext
def rk4_tan_integrator(x,dx,tau,dxdt):
    # 4th order runge-kutta integrator for tangent map of systems with no explicit time dependence
    k1 = dxdt(x,dx)
    k2 = dxdt(x,dx + k1/2*tau)
    k3 = dxdt(x,dx + k2/2*tau)
    k4 = dxdt(x,dx + tau*k3)
    
    dxnext = dx + 1/6*tau*(k1+2*k2+2*k3+k4)
    return dxnext
    
num_elements  = 3     # Dimension of dynamical system
num_exponents = 3     # Number of exponents to be calculated, must be <= dynamical system dimension
tau           = 0.005  # Integration time step, !!MUST BE SMALL!!
iterations    = 20000 # Total number of algorithm iterations
iteration_length = 10 # Number of steps in each iteration before normalization
transient = 5000      # Length of transient

rhos = np.arange(28,49)
lyapunov_times = np.zeros(rhos.size)
for itr, rho in enumerate(rhos):
    dxdt = lambda x:dxdt_lorenz63(x, rho = rho)
    dxdt_tan = lambda x, dx: dxdt_lorenz63_tan(x,dx,rho = rho)
    np.random.seed(10)
    x = np.random.rand(num_elements)
    for i in range(transient):
        x = rk4_integrator(x,tau,dxdt) # Allow initial condition to evolve to attractor

    vecs = np.random.rand(num_elements,num_exponents) # Initialize tangent vectors
    vecs = orth(vecs)

    Rs = np.zeros((num_exponents,iterations))

    # Begin computation
    for iterate in range(iterations):
        for step in range(iteration_length):
            for exponent in range(num_exponents):
                vecs[:,exponent] = rk4_tan_integrator(x,vecs[:,exponent],tau,dxdt_tan) # Compute tangent vector evolution
            x = rk4_integrator(x,tau,dxdt) # Compute vector evolution

        Q,R = np.linalg.qr(vecs) # Find QR factoriation
        Rs[:,iterate] = np.diag(R) # Extract lyapunov numbers over this iteration
        vecs = np.copy(Q) # Set renormalized tangent vectors

    Rs = Rs + 0.0j
    total_time = tau*iterations*iteration_length
    lyapunov_exps = np.real(np.sum(np.log(Rs),axis = 1))/total_time # Calculate exponents
    print(rho)
    print(lyapunov_exps)
    lyapunov_times[itr] = 1/lyapunov_exps[0]

28
[ 9.05248133e-01  4.14931703e-04 -1.45723234e+01]
29
[ 9.26240807e-01 -5.01058640e-04 -1.45924000e+01]
30
[ 9.54018900e-01 -8.27264956e-04 -1.46198518e+01]
31
[ 9.73660682e-01 -8.61762389e-04 -1.46394590e+01]
32
[ 9.91508904e-01  2.72045028e-03 -1.46608893e+01]
33
[ 1.01760616e+00 -2.75020481e-03 -1.46815158e+01]
34
[ 1.03273166e+00  6.94882691e-04 -1.47000864e+01]
35
[ 1.05741388e+00 -3.33668439e-04 -1.47237399e+01]
36
[ 1.06817525e+00  9.75961826e-04 -1.47358109e+01]
37
[ 1.08749773e+00 -3.37177539e-04 -1.47538201e+01]
38
[ 1.11045390e+00 -4.69694804e-04 -1.47766437e+01]
39
[ 1.12637857e+00 -5.31495681e-04 -1.47925065e+01]
40
[ 1.13601664e+00 -3.74060480e-04 -1.48023019e+01]
41
[ 1.15826551e+00 -1.12203328e-03 -1.48238027e+01]
42
[ 1.16422292e+00 -1.54502407e-03 -1.48293371e+01]
43
[ 1.17938414e+00  2.04132881e-04 -1.48462473e+01]
44
[ 1.20698935e+00 -4.72239690e-04 -1.48731761e+01]
45
[ 1.18556253e+00 -9.14264345e-04 -1.48513072e+01]
46
[ 1.24136329e+00  6.92530480e-04 -1.4908714

In [3]:
data = np.hstack((rhos.reshape(-1,1), lyapunov_times.reshape(-1,1)))
np.savetxt('lorenz_lyapunov_times.csv', data, delimiter = ',')